# Test to out-of-the-box MACE potential on LTS and LiMnO

In [53]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

from pymatgen.core.structure import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import read
from ase.data import chemical_symbols
from ase.atoms import Atoms
from ase.visualize import view
import copy
import numpy as np
import pandas as pd
from scipy.constants import physical_constants
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from scipy.stats import spearmanr, kendalltau

import matplotlib.pyplot as plt

import sys
import os
import json
import re
import shutil as sh

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.geom_opt import GeomOpt

current_dir = os.path.dirname(os.path.abspath("__file__"))
sys.path.append(current_dir)

from structure_generation import get_all_configurations_pmg, write_extended_xyz, generate_random_structures, \
    write_CRYSTAL_gui_from_data
from local_functions import lattice_params_to_matrix, get_errors

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read the LTS output files

In [19]:
# Conversion factors
HARTREE_TO_EV = physical_constants['Hartree energy in eV'][0]
BOHR_TO_ANGSTROM = physical_constants['Bohr radius'][0] * 1e10  # Convert meters to Ångstrom
BOHR_CUBED_TO_ANGSTROM_CUBED = BOHR_TO_ANGSTROM**3

def parse_extended_xyz(file_content, num_atoms):
    """
    Parse the file to extract structures and convert lattice parameters, coordinates, energy, forces,
    and stress tensor to standard units (e.g., eV, Å).
    """
    def extract_floats(line):
        """Helper function to extract floats from a string."""
        return list(map(float, re.findall(r"[-+]?\d*\.\d+(?:[Ee][-+]?\d+)?", line)))

    results = []
    structure_data = {}

    for i, line in enumerate(file_content):
        line = line.strip()
        
        # Lattice parameters
        if "ATOM                 X/A                 Y/B                 Z/C" in line:
            lattice_params = extract_floats(file_content[i - 3])
            if len(lattice_params) == 6:
                a, b, c, alpha, beta, gamma = lattice_params
                structure_data['lattice_matrix'] = lattice_params_to_matrix(a, b, c, alpha, beta, gamma)

        # Fractional coordinates and atomic symbols
        if "ATOM                 X/A                 Y/B                 Z/C" in line:
            
            start = i + 2
            fractional_coords = []
            atomic_symbols = []
            for j in range(num_atoms):
                coord_line = file_content[start + j].strip()
                parts = coord_line.split()
                atomic_number = int(parts[2])  # Third element is the atomic number
                atomic_symbols.append(chemical_symbols[atomic_number])  # Convert to symbol
                fractional_coords.append(extract_floats(coord_line))

            structure_data['fractional_coordinates'] = fractional_coords
            structure_data['atomic_symbols'] = atomic_symbols

            # Calculate Cartesian coordinates
            lattice_matrix = structure_data['lattice_matrix']
            structure_data['cartesian_coordinates'] = [
                np.dot(coord, lattice_matrix) for coord in fractional_coords
            ]

        # Energy
        if "== SCF ENDED - CONVERGENCE ON ENERGY      E(AU)" in line:
            energy_hartree = extract_floats(line)[0]
            structure_data['energy_ev'] = energy_hartree * HARTREE_TO_EV

        # Forces
        if "CARTESIAN FORCES IN HARTREE/BOHR (ANALYTICAL)" in line:
            start = i + 2
            structure_data['forces'] = [
                extract_floats(file_content[start + j])
                for j in range(num_atoms)
            ]

        # Stress tensor
        if "STRESS TENSOR, IN HARTREE/BOHR^3:" in line:
            start = i + 4
            stress_hartree_bohr3 = [
                extract_floats(file_content[start + j]) for j in range(3)
            ]
            stress_ev_angstrom3 = np.array(stress_hartree_bohr3) * (HARTREE_TO_EV / BOHR_CUBED_TO_ANGSTROM_CUBED)
            structure_data['stress'] = stress_ev_angstrom3.tolist()

        # Store the structure if all required fields are found
        if all(key in structure_data for key in ['lattice_matrix', 'fractional_coordinates', 'cartesian_coordinates', 'energy_ev', 'forces', 'stress', 'atomic_symbols']):
            results.append(structure_data.copy())
            structure_data = {}  # Reset for the next structure

    return results

In [20]:
with open('data/crystal/lts/output_files/lts_optgeom_full_2009.out', "r") as f:
    file_content = f.readlines()
parse_extended_xyz(file_content, num_atoms=54)

[{'lattice_matrix': array([[11.04220768,  0.        ,  0.        ],
         [ 5.58423713,  9.52610341,  0.        ],
         [ 5.45797056,  3.12713828,  8.99820275]]),
  'fractional_coordinates': [[-0.01251336868143,
    -0.01251336868143,
    0.01251336868143],
   [0.001870111537215, 0.001870111537215, 0.3259000190602],
   [0.001870111537215, 0.001870111537215, -0.3296402421346],
   [-0.01043287451096, 0.3312659375629, 0.01043287451096],
   [-0.004107594319187, 0.3391166940447, 0.3324954501373],
   [-0.01043287451096, 0.3312659375629, -0.3312659375629],
   [-0.01123533898213, -0.3119756960169, 0.01123533898213],
   [-0.01123533898213, -0.3119756960169, 0.3119756960169],
   [-0.02164996790024, -0.3388269696344, -0.3197615312327],
   [0.3312659375629, -0.01043287451096, 0.01043287451096],
   [0.3391166940447, -0.004107594319187, 0.3324954501373],
   [0.3312659375629, -0.01043287451096, -0.3312659375629],
   [0.3197615312327, 0.3197615312327, 0.02164996790024],
   [0.3197615312327, 0.3

In [ ]:
# Write all the extxyz from all output files
import os
import numpy as np
import pandas as pd

# Folder containing the .out files
folder_path = "data/crystal/lts/output_files"
output_folder = "data/crystal/lts/extxyz_files"
os.makedirs(output_folder, exist_ok=True)  # Ensure the output folder exists

file_list = [f for f in os.listdir(folder_path) if f.startswith("lts") and f.endswith(".out")]

global_index = -1
for file_name in file_list:
            
    # Construct file path for the current Z
    file_path = os.path.join(folder_path, file_name)

    # Read the file and process its content
    with open(file_path, "r") as f:
        file_content = f.readlines()

    # Parse the file content
    parsed_structures = parse_extended_xyz(file_content, num_atoms=54)  # Replace 108 with your atom count

    # Convert parsed structures to a DataFrame
    df_structures = pd.DataFrame(parsed_structures)

    # Save all extracted structures with unique global indices
    for _, row in df_structures.iterrows():
        # Generate the output file name with the incrementing global index
        global_index += 1
        output_file = os.path.join(
            output_folder, f"lts_{global_index}.xyz"
        )

        # Write the structure to an extended XYZ file
        with open(output_file, "w") as out_f:
            # Write number of atoms
            num_atoms = len(row['cartesian_coordinates'])
            out_f.write(f"{num_atoms}\n")
            if row['energy_ev']*HARTREE_TO_EV < -55000:
                print('YES',file_name)
            else:
                print('NO',file_name)
            # Write metadata
            lattice_flat = " ".join(f"{value:.12e}" for value in row['lattice_matrix'].flatten())
            stress_flat = " ".join(f"{value:.12e}" for value in np.array(row['stress']).flatten())
            out_f.write(
                f"dft_energy={row['energy_ev']:.12e} "
                f'Lattice="{lattice_flat}" '
                f'dft_stress="{stress_flat}" '
                f'Properties=species:S:1:pos:R:3:dft_forces:R:3 '
                f'config_type=random '
                # f'system_name={os.path.basename(output_file[:-4])}\n'
                f'system_name=random\n'
            )

            # Write atomic data
            for symbol, coord, force in zip(row['atomic_symbols'], row['cartesian_coordinates'], row['forces']):
                out_f.write(
                    f"{symbol} {coord[0]:.12e} {coord[1]:.12e} {coord[2]:.12e} "
                    f"{force[0]:.12e} {force[1]:.12e} {force[2]:.12e}\n"
                )

lts_optgeom_full_1169.out
lts_optgeom_cell_1809.out
lts_optgeom_cell_1809.out
lts_optgeom_cell_1809.out
lts_optgeom_full_4011.out
lts_optgeom_full_4011.out
lts_optgeom_full_4011.out
lts_optgeom_full_4011.out
lts_optgeom_full_4011.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_cell_3978.out
lts_optgeom_full_2312.out
lts_optgeom_full_4005.out
lts_optgeom_full_4005.out
lts_optgeom_full_4005.out
lts_optgeom_full_4005.out
lts_optgeom_full_4005.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_1835.out
lts_optgeom_cell_742.out
lts_optgeom_full_198.out
lts_optgeom_full_198.out
lts_optgeom_full_198.out
lts_optgeom_full_198.out
lts_optgeom_full_198.out
lts_optgeom_full_1

KeyboardInterrupt: 

### Test for duplicates

In [24]:
import os
import hashlib

def calculate_file_hash(file_path, hash_algo="md5"):
    """
    Calculate the hash of a file using the specified algorithm.
    
    Parameters:
        file_path (str): Path to the file.
        hash_algo (str): Hash algorithm to use (default: "md5").
    
    Returns:
        str: Hexadecimal hash of the file content.
    """
    hash_func = hashlib.new(hash_algo)
    with open(file_path, 'rb') as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_func.update(chunk)
    return hash_func.hexdigest()

def find_duplicate_files(folder_path, pattern_prefix):
    """
    Find duplicate files in a folder for a given pattern prefix.
    
    Parameters:
        folder_path (str): Path to the folder containing files.
        pattern_prefix (str): Prefix pattern for filtering files (e.g., "AlGaN_super3_X_Y_").
    
    Returns:
        list of tuple: List of duplicate file pairs (file1, file2).
    """
    # Filter files matching the pattern
    files = [f for f in os.listdir(folder_path) if f.startswith(pattern_prefix)]
    file_hashes = {}
    duplicates = []

    # Calculate hashes for each file
    for file in files:
        file_path = os.path.join(folder_path, file)
        file_hash = calculate_file_hash(file_path)
        if file_hash in file_hashes:
            # Found a duplicate
            duplicates.append((file_hashes[file_hash], file))
        else:
            file_hashes[file_hash] = file

    return duplicates

# Folder containing the .out files
folder_path = "data/crystal/lts/extxyz_files/"


pattern_prefix = f"lts"
duplicates = find_duplicate_files(folder_path, pattern_prefix)

if duplicates:
    print("Duplicate files found:")
    for file1, file2 in duplicates:
        print(f"{file1} and {file2}")
else:
    print("No duplicate files found.")

No duplicate files found.


### Concatenate

In [ ]:
import os

def concatenate_xyz_files(input_folder, output_file):
    """
    Concatenate all .xyz files in a folder into a single .xyz file.

    Parameters:
        input_folder (str): Path to the folder containing .xyz files.
        output_file (str): Path to the output .xyz file.
    """
    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    with open(output_file, 'w') as outfile:
        for file_name in sorted(os.listdir(input_folder)):
            if file_name.endswith(".xyz"):
                file_path = os.path.join(input_folder, file_name)

                # Read the content of the current .xyz file
                with open(file_path, 'r') as infile:
                    content = infile.read()
                
                # Append content to the output file
                outfile.write(content)

    print(f"All .xyz files in '{input_folder}' have been concatenated into '{output_file}'.")

# Example usage
input_folder = "data/crystal/lts/extxyz_files"
output_file = "data/crystal/lts/concatenated_files/lts_all.xyz"
concatenate_xyz_files(input_folder, output_file)

All .xyz files in 'data/crystal/lts/extxyz_files' have been concatenated into 'data/crystal/lts/lts_all.xyz'.


## Compare energies

In [28]:
ase_structure = Atoms(['S'],[[0.0, 0.0, 0.0]])
sp_mace = SinglePoint(
    struct=ase_structure.copy(),
    arch="mace_mp",
    device="cpu",
    model_path="/Users/brunocamino/Desktop/UCL/mace_benchmark/models/mace-mpa-0-medium.model",
    calc_kwargs={"default_dtype": "float64"},
    properties="energy",
)

init_energy = sp_mace.run()["energy"]
print(init_energy)

Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
-0.89160769


In [29]:
li_dft_energy = -7.4535580426043E+00*HARTREE_TO_EV
ti_dft_energy = -8.4917384909564E+02*HARTREE_TO_EV
s_dft_energy = -3.9804168854419E+02*HARTREE_TO_EV

li_mace_energy = -0.29754725
ti_mace_energy = -2.50054871
s_mace_energy = -0.89160769


In [39]:
energies = np.loadtxt("data/lts/pt_model/all_energies.csv", delimiter=",", skiprows=1)

num_sites = 54

# Extract the two columns separately
dft_energies = energies[:, 0]  # First column
mace_energies = energies[:, 1]  # Second column

dft_energies_form = (dft_energies - (li_dft_energy*18 + 
                                    ti_dft_energy*9 + s_dft_energy*27))/num_sites

mace_energies_form = (mace_energies - (li_mace_energy*18 + 
                                    ti_mace_energy*9 + s_mace_energy*27))/num_sites

In [40]:
get_errors(dft_energies_form,mace_energies_form)


MSE: 1958871.854498135
RMSE: 1399.5970328984465
Percentage MSE: 99.80507300596491 %
Max Absolute Error: 4625.884617376744
Max Percentage Error: 99.91605747238552 %


In [49]:
all_structures = read('data/crystal/lts/concatenated_files/lts_all.xyz',":")

In [54]:
view(all_structures[6398])

<Popen: returncode: None args: ['/Users/brunocamino/miniconda3/envs/mace-tes...>

In [67]:
np.sort(dft_energies)

array([-754067.3855685, -742760.6010391, -742685.5482815, ...,
       -504294.3297341, -504292.2282146, -504291.3880448])

In [65]:
len(dft_energies_form)

24032

### Errors

In [46]:
np.argsort(dft_energies_form)

array([ 6398,  9719, 12118, ..., 23261, 15260,  2350])